In [1]:
import pandas as pd
import numpy as np

In [6]:
data = pd.read_csv("churn_analysis.csv")
print(data.shape)
data.head()

(3333, 23)


,Unnamed: 0,state,account_length,area_code,intl_plan,vmail_plan,vmail_message,day_mins,day_calls,day_charge,...,night_mins,night_calls,night_charge,intl_mins,intl_calls,intl_charge,custserv_calls,treatment,mes_estim,churn
0,0,KS,128,415,no,yes,25,265.1,110,45.07,...,244.7,91,11.01,10.0,3,2.70,1,1,0.65,False.
1,1,OH,107,415,no,yes,26,161.6,123,27.47,...,254.4,103,11.45,13.7,3,3.70,1,0,0.55,False.
2,2,NJ,137,415,no,no,0,243.4,114,41.38,...,162.6,104,7.32,12.2,5,3.29,0,0,0.72,False.
3,3,OH,84,408,yes,no,0,299.4,71,50.90,...,196.9,89,8.86,6.6,7,1.78,2,1,0.28,False.
4,4,OK,75,415,yes,no,0,166.7,113,28.34,...,186.9,121,8.41,10.1,3,2.73,3,2,0.45,False.


In [7]:
data_treat1 = data[data.treatment == 1]

In [41]:
ct = pd.crosstab(data_treat1.state, data_treat1.churn)
ct.head()

churn,False.,True.
state,,
AK,19,1
AL,25,5
AR,11,5
AZ,17,2
CA,10,5


In [60]:
from itertools import combinations
from scipy.stats import chi2_contingency

p = []
for a, b in combinations(ct.index, 2):
    subtable = ct.query(f"state == '{a}' | state == '{b}'" )
    chi, pvalue, _, _ = chi2_contingency(subtable, correction=True)
    if (pvalue < 0.05):
        p.append(pvalue)
    
len(p)

0

In [61]:
from scipy.stats import fisher_exact

p = []
for a, b in combinations(ct.index, 2):
    subtable = ct.query(f"state == '{a}' | state == '{b}'" )
    chi, pvalue = fisher_exact(subtable)
    if (pvalue < 0.05):
        p.append(pvalue)
    
len(p)

10

In [65]:
from scipy.stats import pearsonr

a, pvalue = pearsonr(data.day_calls, data.mes_estim)
print(a, pvalue)

-0.051794350587572625 0.0027798836869756707


In [67]:
from scipy.stats import spearmanr

spearmanr(data.day_calls, data.mes_estim)

SpearmanrResult(correlation=0.043349880533927444, pvalue=0.012317367189170541)

In [79]:
import scipy.stats as ss

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    print(ss.chi2_contingency(confusion_matrix)[1])
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [80]:
cramers_v(data_treat1.state, data_treat1.churn)


0.7097590042778473


0.0

In [75]:
ss.chi2_contingency(data_treat1.state, data_treat1.churn)

TypeError: '<' not supported between instances of 'str' and 'int'